In [13]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark= SparkSession.builder.getOrCreate()

sc= spark.sparkContext

sc


<SparkContext master=local[*] appName=pyspark-shell>

In [2]:
import numpy as np
data=np.arange(1,51)

rdd1 =sc.parallelize(data)
rdd2=rdd1.map(lambda x:int(x**2))
rdd3= rdd2.map(lambda x :x+20)
#ممكن اعمل rdd4 يكون السورس بتاعه من 2
rdd4=rdd2.map(lambda x :int(x*2))

In [7]:
# بعرف انا عندي كان برتشن 
rdd2.getNumPartitions()


2

In [8]:
# هنعمل كاش عند rdd2 عشان يحفظ المسار ده عنده 
# لان اغلب العمليات الي عملتها بعد كده جايه عن طريق المسار ده واما بعملها كاش مش بيتفذ الا مره واحده بس 

rdd2.cache()
# محدش بيعمل كوليكت عشان بتاخد وقت كبير لو الداتا كبيره 
               # rdd4.collect()

#هات الو 10 قيم 
rdd4.take(10)

[2, 8, 18, 32, 50, 72, 98, 128, 162, 200]

In [ ]:
# هنجيب الداتا من فايل 
txtRDD =sc.textFile('/home/ahmed-refat/Desktop/text1.txt')

txtRDD.map( lambda l :l.split()).collect()
txtRDD.flatMap( lambda l :l.split()).collect()

txtRDD.filter( lambda line:'f' in line ).collect() #جيب الكلمه الي فيها حرف اف بس 


['first', 'fourth', 'fifth ']

In [16]:
def sq(x):
    return x**2

rdd5=rdd2.map((sq))
rdd5.take(5)

[1, 16, 81, 256, 625]

In [5]:
listname=['ahmed','hadeel','asmaa','mohamed','ali','soliman','khaled']
rddnames=sc.parallelize(listname)
rddnames.collect()

['ahmed', 'hadeel', 'asmaa', 'mohamed', 'ali', 'soliman', 'khaled']

In [6]:
rddgroupby=rddnames.groupBy(lambda w : w[0]) # اعمل مجموعات علي حسب اول حرف في الكلمه 
rddgroupby.collect()




[('h', <pyspark.resultiterable.ResultIterable at 0x7f846c595510>),
 ('s', <pyspark.resultiterable.ResultIterable at 0x7f843003f3d0>),
 ('a', <pyspark.resultiterable.ResultIterable at 0x7f843003d630>),
 ('m', <pyspark.resultiterable.ResultIterable at 0x7f843003efb0>),
 ('k', <pyspark.resultiterable.ResultIterable at 0x7f843003d420>)]

In [37]:
# لو الداتا عندي key and value 

rdd =sc.parallelize([('a',1),('b',1),('a',2),('a',3),('a',4),('a',5)])
rdd_2=rdd.groupByKey()

rdd_2.collect()

[('b', <pyspark.resultiterable.ResultIterable at 0x7255bae12800>),
 ('a', <pyspark.resultiterable.ResultIterable at 0x7255bae138b0>)]

تطبيق عملي
-------------

In [ ]:
  # هنجيب الداتا من الفايل 

rdd_raw =sc.textFile('/home/ahmed-refat/Desktop/text1.txt')
rdd_clean = rdd_raw.filter(lambda line: len(line.split(",")) == 4)

# الماب هتخلي كل سطر عباره عن ليست[]
rdd_parsed = rdd_clean.map(lambda line: line.split(","))

#هنا حولنا البيانات إلى (product, value).
rdd_sales = rdd_parsed.map(lambda x: (x[2], float(x[3])))  # (product, amount)

#reduceByKey() بتجمع القيم اللي ليها نفس المفتاح (هنا المنتج).
rdd_total_sales = rdd_sales.reduceByKey(lambda a, b: a + b)


rdd_total_sales.collect() 
rdd_parsed.collect() 


for product, total in rdd_total_sales.collect():
    print(product, total)



[('Mobile', 3800.0), ('TV', 3000.0), ('Laptop', 10500.0)]